In [1]:
!pip install -U spacy==3.* -q


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m spacy info

C:\Program Files\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!


============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\spacy
Platform         Windows-10-10.0.19042-SP0     


  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
import spacy


Python version   3.9.7                         
Pipelines        en_core_web_sm (3.7.1)        



C:\Program Files\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable

C:\Program Files\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!


     ------------------------                 7.7/12.8 MB 14.1 MB/s eta 0:00:01



  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 94, in read
    self.__buf.write(data)
  File "C:\Program Files\Python39\lib\tempfile.py", line 474, in func_wrapper
    return func(*args, **kwargs)
OSError: [Errno 28] No space 

In [5]:
nlp = spacy.load('en_core_web_sm')


  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_internal\resolution\resolvelib\factory.py", line 206, in _make_candidate_from_link
    self._link_candidate_cache[link] = LinkCandidate(
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_internal\resolution\resolvelib\candidates.py", line 293, in __init__
    super().__init__(
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_internal\resolution\resolvelib\candidates.py", line 156, in __init__
    self.dist = self._prepare()
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_internal\resolution\resolvelib\candidates.py", line 225, in _prepare
    dist = self._prepare_distribution()
  File "C:\Users\raphaella220046\AppData\Roaming\Python\Python39\site-packages\pip\_internal\resolution\resolvelib\candidates.py", line 304, in _prepare_distribution
    return preparer.prepare_linked_requirement(self._ireq, 

In [6]:
type(nlp)

spacy.lang.en.English

In [7]:
import os
import pandas as pd
import numpy as np
from openpyxl import workbook
from openpyxl import load_workbook
import re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import defaultdict

In [8]:
# Defina o diretório onde estão localizados os arquivos .xls
diretorio = 'xls_files/'

# Liste todos os arquivos no diretório
arquivos = os.listdir(diretorio)

# Verifique se já existem arquivos com a extensão .xlsx na pasta
if any(arquivo.endswith('.xlsx') for arquivo in arquivos):
    print("Já existem arquivos com a extensão .xlsx na pasta")
else:
    # Itere sobre cada arquivo
    for arquivo in arquivos:
        # Verifique se o arquivo é um arquivo .xls
        if arquivo.endswith('.xls'):
            # Carregue o arquivo .xls em um DataFrame usando o pandas
            df = pd.read_excel(os.path.join(diretorio, arquivo))

            # Defina o caminho para salvar o arquivo .xlsx com o mesmo nome, mas extensão .xlsx
            novo_nome = os.path.splitext(arquivo)[0] + '.xlsx'
            novo_caminho = os.path.join(diretorio, novo_nome)

            # Salve o DataFrame como um arquivo .xlsx
            df.to_excel(novo_caminho, index=False)

            print(f'O arquivo {arquivo} foi convertido para {novo_nome}')

Já existem arquivos com a extensão .xlsx na pasta


In [9]:
path = "xls_files"
files = os.listdir(path)
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
print(files_xlsx)

['Perovskite and band gap 1-1000.xlsx', 'Perovskite and band gap 1001-2000.xlsx', 'Perovskite and band gap 2001-3000.xlsx', 'Perovskite and band gap 3001-4000.xlsx', 'Perovskite and band gap 4001-4796.xlsx', 'Perovskite and band gap.xlsx']


In [10]:
df_list = []
for f in files_xlsx:
    data = pd.read_excel(os.path.join(path, f))
    df_list.append(data)
df = pd.concat(df_list)

In [11]:
if "Perovskite and band gap.xlsx" in files_xlsx:
    print("Já existe o arquivo na pasta")
else:
    output_path = "xls_files/Perovskite and band gap.xlsx"
    df.to_excel(output_path, index=False)

Já existe o arquivo na pasta


In [12]:
df_perovskite = pd.read_excel("xls_files/Perovskite and band gap.xlsx")
df_perovskite = df_perovskite.dropna(axis = 1, how = 'all')
#print(df_perovskite)

In [13]:
#abstract = df_perovskite["Abstract"].astype(str) #df_perovskite.iloc[:, 21]
df_perovskite['Abstract'] = df_perovskite['Abstract'].astype("string") #apply(str)
print(df_perovskite['Abstract'])

0       Lead-free double-perovskite nanocrystals (NCs)...
1       Perovskite solar cells (PSCs) have attracted u...
2       Perovskite-based double junction tandem solar ...
3       Halide double perovskite has emerged as a nove...
4       The desired morphology, narrowed band gap and ...
                              ...                        
4791    Recently, antimony selenide (Sb2Se3) has been ...
4792    Compared with the first-generation semiconduct...
4793    Semiconductor quantum dots (QDs) are nanocryst...
4794    This review highlights the development of ener...
4795    All-inorganic perovskite quantum dots have att...
Name: Abstract, Length: 4796, dtype: string


In [ ]:
termo = 'band gap'

palavra = "eV" or "electron volt"

inicio = 0
fim = len(df_perovskite['Abstract'])

bg = []
prvskt = []

for abstract in df_perovskite['Abstract'][inicio:fim]:
    sentenca = nlp(abstract)
    abstract = []
    band_gap = []
    for sent in sentenca.sents:
        abstract.append(sent)
        if termo in str(sent):
            matches = re.findall(r'(\d+\.\d+?)\s*' + re.escape(palavra), str(sent))
            band_gap.append(matches)
            for eV in band_gap:
                for token in sent:
                    if str(token) in eV:
                        #\b(?:[A-Z][a-z][A-Z]|[A-Z][A-Z]\d|[A-Z][a-z]\d|[A-Z]\d[A-Z]|[A-Z]{3}\d)\w{3,}\b|\s(\(([A-z]\d.+?))\s
                        match = re.findall(r'\b(?:[A-Z][a-z][A-Z]|[A-Z][A-Z]\d|[A-Z][a-z]\d|[A-Z]\d[A-Z]|[A-Z]{3}\d)\w{3,}\b', ' '.join(str(token) for token in sent))
                        #print(eV, match)
                        bg.append(eV)
                        prvskt.append(match)
                        
print(len(bg))
print(len(prvskt))

In [ ]:
band_gap = []
perovskite = []
                        
for eV, item in zip(bg, prvskt):
    if len(eV) == 1 and len(item) == 1:
        band_gap.append(eV)
        perovskite.append(item)
        #print(band_gap, perovskite)
        PEROVSKITE = [item[0] for item in perovskite]
        BAND_GAP = [eV[0] for eV in band_gap]
        tupla = list(zip(PEROVSKITE, BAND_GAP))
        df = pd.DataFrame(tupla, columns=['Perovskite', 'Band gap (eV)'])
        df.to_csv('Perovskite_bg.csv', sep=',', index=False, encoding='utf-8')
        #print(df)

print(band_gap)
print()
print(perovskite)

In [ ]:
# Conjunto para rastrear os valores únicos em perovskite
valores_unicos = set()

# Listas para armazenar os valores filtrados
novo_band_gap = []
novo_perovskite = []

# Itera sobre as listas band_gap e perovskite
for eV, item in zip(band_gap, perovskite):
    PEROVSKITE = [item[0] for item in perovskite]
    BAND_GAP = [eV[0] for eV in band_gap]
    # Critério de verificação para remover valores repetidos e que não contém um número
    for eV, item in zip(BAND_GAP, PEROVSKITE):
        if item not in valores_unicos and any(char.isdigit() and float(eV)<10 for char in item):
            # Adiciona o item ao conjunto de valores únicos
            valores_unicos.add(item)
            # Adiciona os valores filtrados às novas listas
            novo_band_gap.append(eV)
            novo_perovskite.append(item)

# Cria um DataFrame com os valores filtrados
df = pd.DataFrame({'Perovskite': novo_perovskite, 'Band gap (eV)': novo_band_gap})

# Salva o DataFrame em um arquivo CSV
df.to_csv('Perovskite_bg_tratado.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
# Busca de termo nos abstracts

valor = 'CH3NH3PbI3'#'BiMnO3' #'C5N2H9' #'BaLa4Ti4O' #'PbBi2Nb2'

coluna = 'Abstract'

for resumo in df_perovskite[coluna][inicio:fim]:
    if valor in resumo:#busca == True:
        print(resumo)

In [ ]:
df_pbg = pd.read_csv('Perovskite_bg_tratado.csv', sep=',')
print(df_pbg.shape)
df_pbg.head(10)

In [ ]:
value_min = min(df_pbg['Band gap (eV)'])
element_min = df_pbg.loc[df_pbg['Band gap (eV)'] == value_min, 'Perovskite'].item()
print(f'{value_min} eV {element_min}')
value_max = max(df_pbg['Band gap (eV)'])
element_max = df_pbg.loc[df_pbg['Band gap (eV)'] == value_max, 'Perovskite'].item()
print(f'{value_max} eV {element_max}')

In [ ]:
df_pbg['Band gap (eV)'] = df_pbg['Band gap (eV)'].astype(float)
df_pbg['Band gap (eV)'] = pd.to_numeric(df_pbg['Band gap (eV)'], errors='coerce')
df_pbg = df_pbg.sort_values(by='Band gap (eV)')

# Plotagem do gráfico
plt.rcParams['font.family'] = 'Verdana'
plt.figure(figsize=(20, 40))
plt.plot(df_pbg['Band gap (eV)'], df_pbg['Perovskite'], marker='o', color='green')
plt.xlabel('Band gap (eV)', size=25)
plt.ylabel('Perovskite', size=25)
plt.title('Distribuição de Band gap por Perovskite', size=30)
plt.xticks(rotation=0)
plt.tick_params(labelsize=15)
plt.rcParams['legend.fontsize'] = 15
#plt.grid()
#plt.legent()
plt.tight_layout()
plt.show()
#plt.savefig('...')

In [ ]:
counts, bins = np.histogram(df_pbg['Band gap (eV)'])
plt.hist(bins[:-1], bins, weights=counts)

# Plotagem do histograma
plt.figure(figsize=(10, 10))
sns.histplot(df_pbg['Band gap (eV)'], kde=True, color='blue', bins=10)  # O argumento 'kde=True' adiciona a curva de densidade
plt.xlabel('Band gap (eV)', size=20)
plt.ylabel('Frequência (%)', size=20)
plt.title('Bang gap de perovskitas', size=30)
plt.tick_params(labelsize=20)
plt.savefig('Histogram perovskite band gap', dpi=300)

print("média")
print(np.mean(df_pbg['Band gap (eV)']))
print("desvio padrão")
print(np.std(df_pbg['Band gap (eV)']))
print("mínimo")
print(np.min(df_pbg['Band gap (eV)']))
print("máximo")
print(np.max(df_pbg['Band gap (eV)']))

In [ ]:
# Dicionário para armazenar a contagem de cada palavra
contagem_palavras = defaultdict(int)

# Iterar sobre cada abstract em df_perovskite['Abstract']
for abstract in df_perovskite['Abstract']:
    # Criar um conjunto de palavras únicas no abstract para contar apenas uma vez por abstract
    palavras_abstract = set(abstract.split())
    
    # Iterar sobre cada palavra em df_pbg['Perovskite']
    for palavra in df_pbg['Perovskite']:
        # Verificar se a palavra está presente no abstract
        if palavra in palavras_abstract:
            # Incrementar a contagem da palavra
            contagem_palavras[palavra] += 1
            
palavras = list(contagem_palavras.keys())
print(contagem_palavras)
print()
print(palavras)

In [ ]:
def plot_cloud(wordcloud):
    plt.figure(figsize=(40, 30))  
    plt.imshow(wordcloud)   
    plt.axis("off")
    
# Configura os parâmetros gráficos do wordcloud (dimensões, cores), gera e salva como imagem
wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Pastel1', collocations=False).generate(str(df_word_counts))  
plot_cloud(wordcloud)
plt.savefig('WORDCLOUD-PCD.png')